# Parameters

The next code block sets parameters that are used throughout the remainder of the notebook.

In [1]:
# pylint: disable=invalid-name,missing-module-docstring

# source atlas name
source_atlas = None

# this atlas will be copied to an atlas named projectId_experimentName_sampleSet_polarity_analysisId
# where projectId is JGI Proposal ID Number
#       experiment name is short text description from field 4 (0-indexed) of LCMS filename
#       sampleSet is commonly Pilot, Final - from field 5 (0-indexed) of LCMS filename
#       polarity is 'POS' or 'NEG'
#       analysisId is usernameX where X is the analysis number

# one of 'positive' or 'negative'
polarity = "positive"

# one of 'ISTDsEtc' or 'FinalEMA-HILIC'
output_type = "FinalEMA-HILIC"

# an integer, increment if you need to redo your analysis
# will be appended to your username to create analysis_id
analysis_number = 0

# experiment ID that must match the parent folder containing the LCMS output files
# An example experiment ID is '20201116_JGI-AK_LH_506489_SoilWarm_final_QE-HF_HILICZ_USHXG01530'
experiment = "REPLACE ME"

# Exclude files with names containing any of the substrings in this list. Eg., ['peas', 'beans']
exclude_files = []

# Exclude groups with names containing any of the substrings in this list.
# 'POS' or 'NEG' will be auto-appended later, so you shouldn't use them here.
exclude_groups = ["QC", "InjBl"]

# thresholds for filtering out compounds with weak MS1 signals
num_points = 5
peak_height = 4e5

# include MSMS fragment ions in the output documents?
export_msms_fragment_ions = False

# list of substrings that will group together when creating groups
# this provides additional grouping beyond the default grouping on field #12
groups_controlled_vocab = ["QC", "InjBl", "ISTD"]

# list of tuples contain string with color name and substring pattern.
# Lines in the EIC plot will be colored by the first substring pattern
# that has a match within the name of the hdf5_file. The order they are
# listed in your list is the order they are displayed in the overlays
# (first is front, last is back). Named colors available in matplotlib
# are here: https://matplotlib.org/3.1.0/gallery/color/named_colors.html
# or use hexadecimal values '#000000'. Lines default to black.
line_colors = [("red", "ExCtrl"), ("green", "TxCtrl"), ("blue", "InjBl")]

# The rest of this block contains project independent parameters

# Full path to the directory where you have cloned the metatlas git repo.
# If you ran the 'git clone ...' command in your home directory on Cori,
# then you'll want '/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metatlas'
# where the uppercase letters are replaced based on your NERSC username.
metatlas_repo_path = "/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metatlas"


# Full path to the directory where you want this notebook to store data.
# A subdirectory will be auto created within this directory for each project.
# You can place this anywhere on cori's filesystem, but placing it within your
# global home directory is recommended so that you do not need to worry about
# your data being purged. Each project will take on the order of 100 MB.
project_directory = "/global/homes/FIRST-INITIAL-OF-USERNAME/USERNAME/metabolomics_projects"

# maximum number of CPUs to use
# when running on jupyter.nersc.gov, you are not allowed to set this above 4
max_cpus = 4

# Threshold for how much status information metatlas functions print in the notebook
# levels are 'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'
log_level = "INFO"

In [ ]:
# pylint: disable=wrong-import-position,import-error
import logging  # noqa: E402
import os  # noqa: E402
import sys  # noqa: E402

sys.path.insert(0, metatlas_repo_path)
logger = logging.getLogger("metatlas.jupyter")
logger.debug("sys.executable=%s", sys.executable)
logger.debug("sys.path=%s.", sys.path)
logger.debug("metatlas_repo_path=%s.", metatlas_repo_path)
if not os.path.exists(metatlas_repo_path):
    logging.critical(
        "Directory set for metatlas_repo_path parameter (%s) does not exist or is not accessible.",
        metatlas_repo_path,
    )
    raise ValueError("Invalid metatlas_repo_path parameter in Jupyter Notebook.")
try:
    from metatlas.tools import environment  # noqa: E402

    environment.validate_kernel()
except ModuleNotFoundError as err:
    logging.critical(
        (
            "Could not find metatlas repo at %s. "
            "In the Parameters block, please check the value of metatlas_repo_path."
        ),
        metatlas_repo_path,
    )
    raise ModuleNotFoundError from err
except ImportError as err:
    logging.critical("A newer version of metatlas_repo is required to use this notebook.")
    raise ImportError from err
from metatlas.tools import notebook  # noqa: E402

notebook.setup(log_level)
from metatlas.datastructures import metatlas_dataset as mads  # noqa: E402

%matplotlib widget

In [ ]:
ids = mads.AnalysisIdentifiers(
    source_atlas, experiment, output_type, polarity, analysis_number, project_directory
)

In [ ]:
metatlas_dataset = mads.MetatlasDataset(
    ids, groups_controlled_vocab=groups_controlled_vocab, exclude_files=exclude_files
)

In [ ]:
if metatlas_dataset.ids.output_type in ["FinalEMA-HILIC"]:
    metatlas_dataset.filter_compounds_by_signal(num_points=num_points, peak_height=peak_height)

#### Annotation GUI
If you are re-running this notebook and do not need to make additional changes to RT min/max bounds, then you can skip running the next code cell. Skipping will save you from calculating MSMS hits twice.

In [ ]:
agui = metatlas_dataset.annotation_gui(compound_idx=0, width=15, height=3, colors=line_colors)

In [ ]:
if metatlas_dataset.ids.output_type in ["FinalEMA-HILIC"]:
    metatlas_dataset.filter_compounds_ms1_notes_remove()

In [ ]:
metatlas_dataset.generate_all_outputs()